# This Notebook is for one off code.
### Please always put a markdown to define what your code is doing so we maintain sanity.

Code to drop the columns

In [1]:
%%sql
ALTER TABLE fact_inventory SET TBLPROPERTIES (
   'delta.columnMapping.mode' = 'name',
   'delta.minReaderVersion' = '2',
   'delta.minWriterVersion' = '5'
);

StatementMeta(, 26c0c9c7-eb4f-417f-b47f-7869f92e8512, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
%%sql
SELECT
  CAST(count(*) AS BIGINT) AS max_qty
FROM fact_inventory; --4,583,984,516


StatementMeta(, 8e90a149-7c16-487d-ada2-d7b6e2386e5b, 3, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [6]:
%%sql
ALTER TABLE fact_inventory ALTER COLUMN daily_qty_end_of_day Decimal(18,2)

StatementMeta(, 72fe37a4-def0-455f-b67b-4203dcb2e948, 7, Finished, Available, Finished)

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'Decimal'.(line 1, pos 61)

== SQL ==
ALTER TABLE fact_inventory ALTER COLUMN daily_qty_end_of_day Decimal(18,2)
-------------------------------------------------------------^^^


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format


# Read the CSV file (assuming it's actually a CSV format despite the original `.parquet` call)
df = spark.read.csv(
    'abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Files/dataexample2 (1).csv'
)

# Transform columns to match the expected schema
df_transformed = df.select(
    col("_c0").alias("ATMSEQ"),
    col("_c1").alias("TCLAVE"),
    col("_c2").alias("ICLAVE"),
    col("_c3").alias("PCLAVE"),  # Supplier ID requires calculation from another table
    col("_c4").alias("ATMREF"),
    col("_c5"),
    col("_c6").alias("ATMFECHA"),
    col("_c7").alias("ATMDESC"),
    (-1 * col("_c8")).alias("qty"),  # Convert sales amount to positive
    col("_c10").alias("movement_date"),
    col("_c12").alias("cost"),
    col("_c13").alias("sale")
)

# Display some rows for verification
df_transformed.createTempView('SybaseApril22')


StatementMeta(, 41ffe564-af07-453a-b365-eb3ddb895b85, 16, Finished, Available, Finished)

In [17]:
%%sql
INSERT INTO fmovarti_intermediate_silver (
    seq_per_store, store_id_from, product_id, supplier_id, store_id_to, 
    movement_id, qty, cost_amt, ReferenceDoc, DPFolio, sale_amt, 
    movement_date, src_update_date, fabric_load_date
)
SELECT 
    ATMSEQ,
    TCLAVE AS store_id_from,
    PCLAVE AS product_id,
    ICLAVE AS supplier_id,
    ATMREF AS store_id_to,
    ATMDESC AS movement_id,
    -1* ATMCANT AS qty,
    ATMCOSTO AS cost_amt,
    ATMDOC AS ReferenceDoc,
    DPFOLIO,
    -1* ATMVENTA AS sale_amt,
    ATMFECHA AS movement_date,
    UDDATE AS src_update_date,
    NOW() AS fabric_load_date
FROM fmovarti_hist fh
WHERE ATMFECHA = '2025-01-25'
AND NOT EXISTS (
    SELECT 1 
    FROM fmovarti_intermediate_silver s
    WHERE 
        s.store_id_from = fh.TCLAVE
        AND s.product_id = fh.PCLAVE
        AND s.movement_date = fh.ATMFECHA
        AND s.movement_id = fh.ATMDESC
);


StatementMeta(, c321d388-7a44-42d7-b518-eb4393ff7d78, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
%%sql
UPDATE fmovarti_intermediate_silver
SET 
    sale_amt = sale_amt * -1.0,
    qty = qty * -1.0
WHERE 
    movement_date = DATE('2025-01-25')
    AND DATE(fabric_load_date) = DATE('2025-04-01');

StatementMeta(, 5aaffd8f-6cc1-4daf-8320-c87af66392cd, 8, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>